In [1]:
from __future__ import print_function

import sys
sys.path.append("/Users/abhishekkadian/Documents/Github/jaa-dl/assignment-1")
sys.path.append("/Users/abhishekkadian/Documents/Github/jaa-dl/assignment-1/ladder/ladder-ak/")

import numpy as np
import pickle

import torch
from torch.autograd import Variable
from torch.optim import Adam

import ladder
import encoder
import decoder

from matplotlib import pyplot as plt
import random

%matplotlib inline

In [19]:
ladder = reload(ladder)

In [3]:
def evaluate_decoder_grad(temp_decoder):
    def sum_grad(v):
        return torch.sum(torch.abs(v))
    gg = sum_grad(temp_decoder.a1.grad) + sum_grad(temp_decoder.a2.grad) + sum_grad(temp_decoder.a3.grad) + \
            sum_grad(temp_decoder.a4.grad) + sum_grad(temp_decoder.a5.grad) + \
            sum_grad(temp_decoder.a6.grad) + sum_grad(temp_decoder.a7.grad) + sum_grad(temp_decoder.a8.grad) + \
            sum_grad(temp_decoder.a9.grad) + sum_grad(temp_decoder.a10.grad)
    return gg

In [2]:
batch_size = 32

with open("data/train_labeled.p") as f:
    train_dataset = pickle.load(f)
with open("data/train_unlabeled.p") as f:
    unlabeled_dataset = pickle.load(f)
unlabeled_dataset.train_labels = torch.LongTensor(
    [-1 for x in range(unlabeled_dataset.train_data.size()[0])])
with open("data/validation.p") as f:
    valid_dataset = pickle.load(f)
    
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
unlabeled_loader = torch.utils.data.DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded
3000
750


In [15]:
train_labelled_data = []
for data, target in train_loader:
    train_labelled_data.append((data, target))

In [20]:
noise_std = 0.2
epochs = 1

encoder_in = 28 * 28
decoder_in = 10
encoder_sizes = [1000, 500, 250, 250, 250, decoder_in]
decoder_sizes = [250, 250, 250, 500, 1000, encoder_in]
unsupervised_costs_lambda = [0.1, 0.1, 0.1, 0.1, 0.1, 20., 2000.]

encoder_activations = ["relu", "relu", "relu", "relu", "relu", "softmax"]
encoder_train_bn_scaling = [False, False, False, False, False, True]
encoder_bias = [False, False, False, False, False, False]

mnist_ladder = ladder.Ladder(encoder_in, encoder_sizes, decoder_in, decoder_sizes, encoder_in,
                encoder_activations, encoder_train_bn_scaling, encoder_bias, noise_std)

optimizer = Adam(mnist_ladder.parameters(), lr=0.002)
loss_labelled = torch.nn.CrossEntropyLoss()
loss_unsupervised = torch.nn.MSELoss()

grad_decoder_a_updates = []
history_unsupervised_cost = []

updated code


In [ ]:
for e in range(epochs):
    agg_cost = 0.
    agg_supervised_cost = 0.
    agg_unsupervised_cost = 0.
    num_batches = 0

    # Training
    mnist_ladder.train()

    ind_labelled = 0

    for batch_idx, (unlabeled_data, unlabeled_target) in enumerate(unlabeled_loader):
        # Labelled data
        if ind_labelled == len(train_labelled_data):
            # reset counter
            random.shuffle(train_labelled_data)
            ind_labelled = 0
        labelled_data = train_labelled_data[ind_labelled][0]
        labelled_target = train_labelled_data[ind_labelled][1]
        ind_labelled += 1

        labelled_data_size = labelled_data.size()[0]

        data = np.concatenate((labelled_data.numpy(), unlabeled_data.numpy()), axis=0)
        target = np.concatenate((labelled_target.numpy(), unlabeled_target.numpy()), axis=0)

        data = torch.FloatTensor(data)
        target = torch.LongTensor(target)

        data = data[:,0,:,:].numpy()
        data = data.reshape(data.shape[0], 28 * 28)
        data = torch.FloatTensor(data)
        # TODO: Hold off on this, things should work right now because LongTensor is only used for cost.
        # TODO: Change from LongTensor to FloatTensor. Autograd has a bug with LongTensor.
        data, target = Variable(data), Variable(target)

        # Pass data through the network

        optimizer.zero_grad()

        # do a noisy pass
        output_noise = mnist_ladder.forward_encoders_noise(data)
        tilde_z_layers = mnist_ladder.get_encoders_tilde_z(reverse=True)

        # do a clean pass
        _ = mnist_ladder.forward_encoders_clean(data)
        z_pre_layers = mnist_ladder.get_encoders_z_pre(reverse=True)
        z_layers = mnist_ladder.get_encoders_z(reverse=True)

        # pass through decoders
        hat_z_layers = mnist_ladder.forward_decoders(tilde_z_layers, output_noise)

        z_pre_layers.append(data)

        # TODO: Verify if you have to batch-normalize the bottom-most layer also

        # batch normalizing the image also for cost comparison
        bn_data = data
#         bn_data = mnist_ladder.bn_input_image(data)
        z_layers.append(bn_data)

        # batch normalize using mean, var of z_pre
        bn_hat_z_layers = mnist_ladder.decoder_bn_hat_z_layers(hat_z_layers, z_pre_layers)

        assert len(z_layers) == len(bn_hat_z_layers)

        # calculate costs
        cost_supervised = loss_labelled.forward(output_noise[:labelled_data_size], target[:labelled_data_size])
        cost_unsupervised = 0.
        u_cost_history = []
        for cost_lambda, z, bn_hat_z in zip(unsupervised_costs_lambda, z_layers, bn_hat_z_layers):
            c = cost_lambda * loss_unsupervised.forward(bn_hat_z, z)
            u_cost_history.append(c.data[0])
            cost_unsupervised += c
        history_unsupervised_cost.append(u_cost_history)

        # backprop
        cost = cost_supervised + cost_unsupervised
        cost.backward()

        agg_cost += cost.data[0]
        agg_supervised_cost += cost_supervised.data[0]
        agg_unsupervised_cost += cost_unsupervised.data[0]

        optimizer.step()

        num_batches += 1
        
#         print("BREAKING EARLY")
#         break

        if ind_labelled == len(train_labelled_data):
            # Evaluation
            mnist_ladder.eval()
            ladder.evaluate_performance(mnist_ladder, agg_cost, agg_supervised_cost, agg_unsupervised_cost, 
                                        num_batches, valid_loader, e, ind_labelled)
            # reset costs
            agg_cost = 0.
            agg_supervised_cost = 0.
            agg_unsupervised_cost = 0.
            num_batches = 0
            mnist_ladder.train()
            
            print("BREAKING BAD")
            break

    # Evaluation
#     mnist_ladder.eval()
#     ladder.evaluate_performance(mnist_ladder, agg_cost, agg_supervised_cost, agg_unsupervised_cost,
#                                 num_batches, valid_loader, e, ind_labelled)
#     mnist_ladder.train()

In [10]:
test_bn = torch.nn.BatchNorm1d(28 * 28, affine=False)

In [11]:
all(np.isclose(test_bn(data)[0].data.numpy(), z_layers[-1][0].data.numpy()))

True

In [18]:
history_unsupervised_cost

[[0.16994230449199677,
  0.1520533412694931,
  0.1463761180639267,
  0.1488255113363266,
  0.1513332575559616,
  30.847654342651367,
  13232826.0],
 [0.16386182606220245,
  0.15012489259243011,
  0.14290332794189453,
  0.14446774125099182,
  0.14567382633686066,
  28.7259521484375,
  12357650.0],
 [0.15061624348163605,
  0.14993156492710114,
  0.14054004848003387,
  0.14280438423156738,
  0.1462474912405014,
  28.913280487060547,
  12149988.0],
 [0.15210600197315216,
  0.14606966078281403,
  0.14262360334396362,
  0.14077530801296234,
  0.14304779469966888,
  29.01106834411621,
  12292043.0],
 [0.1483362466096878,
  0.14977838099002838,
  0.14246797561645508,
  0.1424437314271927,
  0.14406244456768036,
  28.421552658081055,
  11945292.0],
 [0.1515161246061325,
  0.1453167051076889,
  0.14033059775829315,
  0.13503269851207733,
  0.13935911655426025,
  27.929964065551758,
  12456930.0],
 [0.148350790143013,
  0.14249779284000397,
  0.13850311934947968,
  0.13731789588928223,
  0.135385